# Lab 2
## Medical Patient No Show

### Team members: Luay Dajani, Dana Geislinger, Chris Morgan, Caroll Rodriguez
##### Github - https://github.com/cdmorgan103/7331DataMiningNoShow

MSDS 7331, 10/28/2018

In [1]:
import numpy as np
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
from pprint import pprint
from IPython.display import display
import seaborn as sns; sns.set(font_scale=1.2)
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit 
import datetime 
from sklearn.model_selection import TimeSeriesSplit


# Hide deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Load the data into variable 'df' from pickled object
from funcs import load_df
df = load_df()

## Data Preparation Part 1
#### [10 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

**Meetings Notes:** Chris, Luay, Caroll - Add Dimensionality reduction and scaling from Mini Lab.
Variable to remove - AppointmentID & PatientID
Luay to add dimensionality reduction

In [2]:
#set AppointmentDay as index so that when train, test is split according to time series
df = df.set_index('AppointmentDay')

In [3]:
#Remove attributes not usefull
del df['PatientId']
del df['AppointmentID']

# perform one-hot encoding of the categorical data 
tmp_df = pd.get_dummies(df.Handicap,prefix='Handicap')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.AppointmentDOW,prefix='AppointmentDOW')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.ScheduledDOW,prefix='ScheduledDOW')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.Neighbourhood,prefix='Neighbourhood')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

tmp_df = pd.get_dummies(df.age_range,prefix='age_range')
df = pd.concat((df,tmp_df),axis=1) # add back into the dataframe

# replace the current Gender attribute with something slightly more intuitive and readable
df['IsMale'] = df.Gender=='M' 
df.IsMale = df.IsMale.astype(np.int)

# Now let's clean up the dataset
if 'Gender' in df:
    del df['Gender'] # if 'Sex' column still exists, delete it (as we created an ismale column)
    
if 'Handicap' in df:    
    del df['Handicap'] # get rid of the original category as it is now one-hot encoded
    
if 'ScheduledDOW' in df:    
    del df['ScheduledDOW'] # get rid of the original category as it is now one-hot encoded
    
if 'AppointmentDOW' in df:    
    del df['AppointmentDOW'] # get rid of the original category as it is now one-hot encoded

if 'Neighbourhood' in df:    
    del df['Neighbourhood'] # get rid of the original category as it is now one-hot encoded

if 'age_range' in df:
    del df['age_range']

if 'AppointmentID' in df:
    del df['AppointmentID']
    
#if 'ScheduledDay' in df:
   #del df['ScheduledDay']
    
#if 'ScheduledTime' in df:
    #del df['ScheduledTime']

#if 'AppointmentDay' in df:
 #   del df['AppointmentDay']

# Get an overview of the raw data
df.info(null_counts=True, verbose=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 110527 entries, 2016-04-29 to 2016-06-07
Data columns (total 114 columns):
ScheduledDay                                 110527 non-null datetime64[ns]
ScheduledTime                                110527 non-null object
Age                                          110527 non-null int64
Scholarship                                  110527 non-null int64
Hypertension                                 110527 non-null int64
Diabetes                                     110527 non-null int64
Alcoholism                                   110527 non-null int64
SMSReceived                                  110527 non-null int64
NoShow                                       110527 non-null bool
DaysInAdvance                                110527 non-null int64
Handicap_0                                   110527 non-null uint8
Handicap_1                                   110527 non-null uint8
Handicap_2                                   110527 non-nul

In [4]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 110527 entries, 2016-04-29 to 2016-06-07
Data columns (total 114 columns):
ScheduledDay                                 datetime64[ns]
ScheduledTime                                object
Age                                          int64
Scholarship                                  int64
Hypertension                                 int64
Diabetes                                     int64
Alcoholism                                   int64
SMSReceived                                  int64
NoShow                                       bool
DaysInAdvance                                int64
Handicap_0                                   uint8
Handicap_1                                   uint8
Handicap_2                                   uint8
Handicap_3                                   uint8
Handicap_4                                   uint8
AppointmentDOW_Monday                        uint8
AppointmentDOW_Tuesday                       ui

In [7]:
df.head(10)

,ScheduledDay,ScheduledTime,Age,Scholarship,Hypertension,Diabetes,Alcoholism,SMSReceived,NoShow,DaysInAdvance,...,Neighbourhood_SÃO CRISTÓVÃO,Neighbourhood_SÃO JOSÉ,Neighbourhood_SÃO PEDRO,Neighbourhood_TABUAZEIRO,Neighbourhood_UNIVERSITÁRIO,Neighbourhood_VILA RUBIM,age_range_child,age_range_adult,age_range_senior,IsMale
AppointmentDay,,,,,,,,,,,,,,,,,,,,,
2016-04-29,2016-04-29,18:38:08,62,0,1,0,0,0,False,0,...,0,0,0,0,0,0,0,1,0,0
2016-04-29,2016-04-29,16:08:27,56,0,0,0,0,0,False,0,...,0,0,0,0,0,0,0,1,0,1
2016-04-29,2016-04-29,16:19:04,62,0,0,0,0,0,False,0,...,0,0,0,0,0,0,0,1,0,0
2016-04-29,2016-04-29,17:29:31,8,0,0,0,0,0,False,0,...,0,0,0,0,0,0,1,0,0,0
2016-04-29,2016-04-29,16:07:23,56,0,1,1,0,0,False,0,...,0,0,0,0,0,0,0,1,0,0
2016-04-29,2016-04-27,08:36:51,76,0,1,0,0,0,False,2,...,0,0,0,0,0,0,0,0,1,0
2016-04-29,2016-04-27,15:05:12,23,0,0,0,0,0,True,2,...,0,0,0,0,0,0,0,1,0,0
2016-04-29,2016-04-27,15:39:58,39,0,0,0,0,0,True,2,...,0,0,0,0,0,0,0,1,0,0
2016-04-29,2016-04-29,08:02:16,21,0,0,0,0,0,False,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
# we want to predict the X and y data as follows:
if 'NoShow' in df:
    y = df['NoShow'].values # get the labels we want
    df_x = df.copy()
    del df_x['NoShow'] # get rid of the class label
    X = df_x.values # use everything else to predict!
    
    
print ('Number of Prediciting Features: ', X.shape)
print ('Number of Response: ', y.shape)

Number of Prediciting Features:  (110527, 113)
Number of Response:  (110527,)


## Data Preparation Part 2
#### [5 points] Describe the ﬁnal dataset that is used for classiﬁcation/regression (include a description of any newly formed variables you created).

**Meeting Notes:** Luay wants add a field that ranks the patient's no show history. If they showed up twice and didn't show up 1 would be 33%. Create a temp dataset to group Patient and determine the # of no shows then merge the dataset and build the ranking. 

https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

## Modeling and Evaluation 1

#### [10 points] Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

**Meeting Notes:** Was in Lab1. Chris wants to add a better explaination of accuracy.

## Modeling and Evaluation 2	

#### [10 points]	Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

**Meeting Notes:** Caroll to find code for splitting time series data.

##### Method 1 - split out the last month of data as test set

In [ ]:
#Testing set --- hold 1 month of data out as test set based on AppointmentDay
df_test = df.copy()
X_test = df_test[(df_test['AppointmentDay'] >= datetime.date(2016,5,1))]
y_test = df_test['NoShow'].values # get the labels we want.
del X_test['NoShow'] # get rid of the class label


#Training set -- all records less than appointment date 5/1/2016
df_train = df.copy()
X_train = df_train[(df_x['AppointmentDay'] < datetime.date(2016,5,1))]
y_train = df_train['NoShow'].values # get the labels we want.
del X_train['NoShow'] # get rid of the class label

In [ ]:
#view first 10 rows of Training Set
X_train.head(10)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [ ]:
df['NoShow'] = df['NoShow']
df['NoShow_num'] = np.where(df['NoShow']==True, 1, 0)

#Set dataframe with index as Appointment Day
ts = df[['AppointmentDay', 'NoShow'] ]
ts = df.set_index('AppointmentDay')
ts.groupby(['AppointmentDay'])['NoShow'].sum()

Plot below shows the # of NoShow appointmets over time.

In [ ]:
plt.plot(ts.groupby(['AppointmentDay'])['NoShow'].sum())

## Modeling and Evaluation 3

#### [20 points] Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

**Meeting Notes**: 1. Random Forest 2. SVM (radial) maybe and 3. Logistic
Chris to ask Dr Drew about clarification of which models we can use. SVM as radial. Check Naive Bayes is an option

## Modeling and Evaluation 4

#### [10 points] Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.

**Meeting Notes**  Precision, recall, F-measure, ROC, AUC

## Modeling and Evaluation 5

#### [10 points] Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course

## Modeling and Evaluation 6

#### [10 points] Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

## Deployment (5 points total) 

#### How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? 

## Exceptional Work (10 points total) 

#### You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?

**Meeting Notes** - We did this in MiniLab 1 with chart 
Possible to add Naives Bayes classification (Luay)